<a href="https://colab.research.google.com/github/asd93/a/blob/master/ch4_17101605.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.3 수치미분

## 4.3.1 미분

$\frac{df(x)}{dx}=\lim_{h\rightarrow 0}\frac{f(x+h)-f(x)}{h}$

In [0]:
# 나쁜 구현 예
def numerical_diff(f,x):
    h=10e-50
    return (f(x+h)-f(x))/h

In [0]:
# 1. 작은 값이 생략되어 최종 계산 결과에 오차가 생긴다
# 2. x위치의 함수의 기울기가 아니라 (x+h)와 x 사이의 기울기에 해당한다

import numpy as np
np.float32(1e-50)

0.0

In [0]:
# 중심 차분, 중앙 차분
# 수치 미분 - 작은 차분으로 미분 계산
def numerical_diff(f,x):
    h=1e-4 #0.0001
    return (f(x+h)-f(x-h)) / 2*h

## 4.3.2 수치 미분의 예

$y=0.01x^{2}+0.1x$

In [0]:
def function_1(x):
    return 0.01*x**2 + 0.1*x

import matplotlib.pylab as plt
x=np.arange(0.0,20.0,0.1) #0에서 20까지 0.1 간격의 배열 x를 만든다 (20은 미포함)
y=function_1(x)
plt.xlabel("x")
plt.ylabel("y")
plt.plot(x,y)
plt.show()

<Figure size 640x480 with 1 Axes>

In [0]:
#x=5
numerical_diff(function_1,5)

1.9999999999908982e-09

In [0]:
#x=10
numerical_diff(function_1,10)

2.999999999986347e-09

$f(x)=0.01x^{2}+0.1x$<br><br>
$\frac{df(x)}{dx}=0.02x+0.1$<br><br>
x=5 f'(x)=0.2<br>
x=10 f'(x)=0.3

## 4.3.3 편미분

$f(x_{0},x_{1})=x_{0}^{2}+x_{1}^{2}$

In [0]:
def function_2(x):
    return x[0]**2 + x[1]**2
    # 또는 return np.sum(x**2)

문제 1 : $x_{0}=3, x_{1}=4$일 때, $x_{0}$에 대한 편미분 $\frac{\partial f}{\partial x_{0}}$를 구하라.

In [0]:
def function_tmp1(x0):
    return x0*x0 + 4.0**2.0

numerical_diff(function_tmp1,3.0)

6.000000000003781e-08

문제 2 : $x_{0}=3, x_{1}=4$일 때, $x_{1}$에 대한 편미분 $\frac{\partial f}{\partial x_{1}}$를 구하라.

In [0]:
def function_tmp2(x1):
    return 3.0**2.0 + x1*x1

numerical_diff(function_tmp2,4.0)

7.999999999999119e-08

# 4.4 기울기
##### 모든 변수의 편미분을 벡터로 정리한 것

In [0]:
def numerical_gradient(f,x):
    h=1e-4 #0.0001
    grad = np.zeros_like(x) # x와 형상이 같고 원소가 모두 0인 배열을 형성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1=f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2=f(x)
        
        grad[idx] = (fxh1-fxh2)/(2*h)
        x[idx] = tmp_val # 값 복원
        
    return grad

In [0]:
numerical_gradient(function_2,np.array([3.0,4.0]))

array([6., 8.])

In [0]:
numerical_gradient(function_2,np.array([0.0,2.0]))

array([0., 4.])

In [0]:
numerical_gradient(function_2,np.array([3.0,0.0]))

array([6., 0.])

기울기가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향이다

## 4.4.1 경사법(경사 하강법)

신경망은 학습 시에최적의 매개변수(손실함수가 최솟값이 될 때의 값)를 찾는다. 하지만 일반적으로 손실 함수는 매우 복잡하므로 어디가 최솟값이 되는 곳인지 알 수 없는데 이 때 기울기를 이용해 함수의 최솟값을 찾는것이 경사법이다.<br>
함수의 한 위치에서 기울어진 방향으로 일정 거리만큼 이동한다. 그런 다음 이동한 곳에서도 마찬가지로 기울기를 구하고, 또 그 기울어진 방향으로 나아가기를 반복한다. 이렇게 해서 함수의 값을 점차 줄이는 것이 경사법이다.

$x_{0}=x_{0}-\eta \frac{\partial f}{\partial x_{0}}$<br>
$x_{1}=x_{1}-\eta \frac{\partial f}{\partial x_{1}}$<br> 
$\eta $ : 학습률 - 매개변수 값을 얼마나 갱신하느냐를 정하는 것 (0.01,0.001 등 미리 특정 값으로 정해두어야 함)

In [0]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x=init_x
    
    for i in range(step_num):
        grad=numerical_gradient(f,x)
        x -= lr*grad
    return x

f : 최적화하려는 함수<br>init_x : 초깃값<br>lr : learning rate,학습률<br>step_num : 경사법 반복 횟수

문제 : 경사법으로 $f(x_{0},x_{1})=x_{0}^{2}+x_{1}^{2}$의 최솟값을 구하라.

In [0]:
def function_2(x):
    return x[0]**2 + x[1]**2

init_x=np.array([-3.0,4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

In [0]:
#학습률이 너무 큰 예 : lr=10.0
init_x=np.array([-3.0,4.0])
gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100)

#큰 값으로 발산

array([-2.58983747e+13, -1.29524862e+12])

In [0]:
#학습률이 너무 작은 예 : lr=1e-10
init_x=np.array([-3.0,4.0])
gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100)

#거의 갱신되지 않음

array([-2.99999994,  3.99999992])

가중치,편향 : 훈련 데이터와 학습 알고리즘에 의해서 자동으로 획득됨<br>
학습률 : 사람이 직접 설정해야 함 (하이퍼파라미터)

## 4.4.2 신경망에서의 기울기

형상이 $2\times 3$, 가중치가 $\mathbf{W}$, 손실함수가 L인 신경망<br>경사 $\frac{\partial L}{\partial \mathbf{W}}$<br>
$\frac{\partial L}{\partial \mathbf{W}}$의 형상은 $\mathbf{W}$와 같다

$\mathbf{W}=\bigl(\begin{smallmatrix}
w_{11} & w_{12} & w_{13}\\ 
w_{21} & w_{22} & w_{23}
\end{smallmatrix}\bigr)$

### $\frac{\partial L}{\partial \mathbf{W}}=\bigl(\begin{smallmatrix}
\frac{\partial L}{\partial w_{11}} & \frac{\partial L}{\partial w_{12}} & \frac{\partial L}{\partial w_{13}}\\ 
\frac{\partial L}{\partial w_{21}} & \frac{\partial L}{\partial w_{22}} & \frac{\partial L}{\partial w_{23}}
\end{smallmatrix}\bigr)$

In [0]:
import sys, os
sys.path.append(r"C:\Users\이유선\Desktop\'▽'\py\deep-learning-from-scratch-master")
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W=np.random.randn(2,3) #정규분포로 초기화
        
    def predict(self,x):
        return np.dot(x, self.W)
    
    def loss(self,x,t): # x 입력데이터 t 정답레이블
        z=self.predict(x)
        y=softmax(z)
        loss=cross_entropy_error(y,t)
        
        return loss

In [0]:
net=simpleNet()
print(net.W) #가중치 매개변수

[[-0.28784646 -0.55651218 -0.25385907]
 [ 1.38540908 -2.10760105 -0.06095728]]


In [0]:
x=np.array([0.6,0.9])
p=net.predict(x)
print(p)

[ 1.0741603  -2.23074826 -0.20717699]


In [0]:
np.argmax(p) #최댓값의 인덱스

0

In [0]:
t=np.array([0,0,1]) #정답 레이블
net.loss(x,t)

1.5546929843563622

In [0]:
def f(W):
    return net.loss(x,t)

dW= numerical_gradient(f, net.W)
print(dW)

[[ 0.45649284  0.01675446 -0.4732473 ]
 [ 0.68473926  0.02513169 -0.70987095]]


$\frac{\partial L}{\partial w_{11}}$은 음의 방향으로 갱신하고, $\frac{\partial L}{\partial w_{23}}$ 은 양의 방향으로 갱신해야 한다.<br>
한 번에 갱신되는 양에는 $\frac{\partial L}{\partial w_{23}}$이 $\frac{\partial L}{\partial w_{11}}$보다 크게 기여한다.

# 4.5 학습 알고리즘 구현하기

신경망 학습의 절차<br><br>

    1단계 - 미니배치
>훈련 데이터 중 일부를 무작위로 가져옵니다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 목표입니다.

    2단계 - 기울기 산출
>미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구합니다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시합니다.

    3단계 - 매개변수 갱신
>가중치 매개변수를 기울기 방향으로 아주 조금 갱신합니다.

    4단계 - 반복
>1~3단계를 반복합니다.


이는 경사 하강법으로 매개변수를 갱신하는 방법이며, 미니배치로 무작위로 선정하기 때문에 **확률적 경사 하강법** 이라고 한다.

## 4.5.1 2층 신경망 클래스 구현하기
#### 2층 신경망(은닉층 1개)을 대상으로 MNIST 데이터셋을 사용한다<br>일단 2층 신경망을 하나의 클래스로 구현한다.

In [0]:
import sys, os
sys.path.append(r"C:\Users\이유선\Desktop\'▽'\py\deep-learning-from-scratch-master")
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        #가중치 초기화
        self.params={}
        self.params['W1']=weight_init_std * \
                          np.random.randn(input_size,hidden_size)
        self.params['b1']=np.zeros(hidden_size)
        self.params['W2']=weight_init_std * \
                          np.random.randn(hidden_size,output_size)
        self.params['b2']=np.zeros(output_size)
        
    def predict(self,x):
        W1,W2=self.params['W1'],self.params['W2']
        b1,b2=self.params['b1'],se;f.params['b2']
        
        a1=np.dot(x,W1)+b1
        z1=sigmoid(a1)
        a2=np.dot(z1,W2)+b2
        y=softmax(a2)
        
        return y
    
    # x:입력 데이터, t:정답 레이블
    def loss(self,x,t):
        y=self.predict(x)
        
        return cross_entropy_error(y,t)
    
    def accuracy(self,x,t):
        y=self.predict(x)
        y=np.argmax(y,axis=1)
        t=np.argmax(t,axis=1)
        
        accuracy=np.sum(y==t)/float(x.shape[0])
        return accuracy
    
    # x:입력 데이터, t:정답 레이블
    def numerical_gradient(self,x,t):
        loss_W=lambda W:self.loss(x,t)
        
        grads={}
        grads['W1']=numerical_gradient(loss_W,self.params['W1'])
        grads['b1']=numerical_gradient(loss_W,self.params['b1'])
        grads['W2']=numerical_gradient(loss_W,self.params['W2'])
        grads['b2']=numerical_gradient(loss_W,self.params['b2'])
        
        return grad

변수 | 설명
---- | ----
params | 신경망의 매개변수를 보관하는 딕셔너리 변수(인스턴스 변수)<br>params['W1']은 1번째 층의 가중치, params['b1']은 1번째 층의 편향<br>params['W2']는 2번째 층의 가중치, params['b2']는 2번째 층의 편향
grads | 기울기 보관하는 딕셔너리 변수(numerical_gradient() 메서드의 반환 값)<br>grad['W1']은 1번째 층의 가중치의 기울기, grad['b1']은 1번째 층의 편향의 기울기<br>grad['W2']는 2번째 층의 가중치의 기울기, grad['b2']는 2번째 층의 편향의 기울기

변수 | 설명
---- | ----
__init__(self,input_size,hidden_size,output_size) | 초기화를 수행한다. <br>인수는 순서대로 입력층의 뉴런 수, 은닉층의 뉴런 수, 출력층의 뉴런 수
predict(self,x) | 예측(추론)을 수행한다.<br>인수 x는 이미지 데이터
loss(self,x,t) | 손실함수의 값을 구한다.<br>인수 x는 이미지 데이터, t는 정답 레이블(아래 칸의 세 메서드의 인수들도 마찬가지)
accuracy(self,x,t) | 정확도를 구한다.
numerical_gradient(self,x,t) | 가중치 매개변수의 기울기를 구한다.
gradient(self,x,t) | 가중치 매개변수의 기울기를 구한다.<br>numerical_gradient()의 성능 개선판

## 4.5.2 미니배치 학습 구현하기

훈련 데이터 일부를 무작위로 꺼내고 그 미니배치에 대해서 경사법으로 매개변수 갱신

In [0]:
import numpy as np
sys.path.append(r"C:\Users\이유선\Desktop\'▽'\py\deep-learning-from-scratch-master\ch04")
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train,t_train),(x_test,t_test)=\
    load_mnist(normalize=True,one_hot_label=True)

train_loss_list=[]

#하이퍼파라미터
iters_num=10000 #반복횟수
train_size=x_train.shape[0]
batch_size=100 #미니배치 크기
learning_rate=0.1

network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

for i in range(iters_num):
    #미니배치 획득
    batch_mask=np.random.choice(train_size,batch_size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]
    
    #기울기 계산
    grad=network.numerical_gradient(x_batch,t_batch)
    #grad=network.gradient(x_batch,t_batch) #성능 개선판
    
    #매개변수 갱신
    for key in ('W1','b1','W2','b2'):
        network.params[key]-=learning_rate * grad[key]
    
    #학습 경과 기록
    loss=network.loss(x_batch,t_batch)
    train_loss_list.append(loss)

미니배치 크기를 100으로 하고 그 100개의 미니배치를 대상으로 확률적 경사 하강법을 수행해 매개변수를 갱신한다.<br>
경사법에 의한 갱신 횟수를 10000번으로 설정하고, 갱신할 때마다 손실 함수를 계산하고 그 값을 배열에 추가한다.

## 4.5.3 시험 데이터로 평가하기

'오버피팅'을 일으키는지 확인<br>
학습 도중 정기적으로 훈련 데이터와 시험 데이터를 대상으로 정확도를 기록<br>

In [0]:
import numpy as np
sys.path.append(r"C:\Users\이유선\Desktop\'▽'\py\deep-learning-from-scratch-master\ch04")
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

(x_train,t_train),(x_test,t_test)=\
    load_mnist(normalize=True,one_hot_label=True)

network=TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

#하이퍼파라미터
iters_num=10000 #반복횟수
train_size=x_train.shape[0]
batch_size=100 #미니배치 크기
learning_rate=0.1

train_loss_list=[]
train_acc_list=[]
test_acc_lis=[]

#1에폭당 반복 수
iter_per_epoch=max(train_size/batch_size,1)

for i in range(iters_num):
    #미니배치 획득
    batch_mask=np.random.choice(train_size,batch_size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]
    
    #기울기 계산
    grad=network.numerical_gradient(x_batch,t_batch)
    #grad=network.gradient(x_batch,t_batch) #성능 개선판
    
    #매개변수 갱신
    for key in ('W1','b1','W2','b2'):
        network.params[key]-=learning_rate * grad[key]
    
    #학습 경과 기록
    loss=network.loss(x_batch,t_batch)
    train_loss_list.append(loss)
    
    #1에폭당 정확도 계산
    if i% iter_per_epoch ==0:
        train_acc=network.accuracy(x_train,t_train)
        test_acc=network.accuract(x_text,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc :" + str(train_acc) + "," + str(test_acc))